In [ ]:

#C:\Users\s1160503\Downloads\ShinyGUI.ipynb

import dash
from dash import dcc, html, Input, Output, State, dash_table
from dash.dash_table.Format import Format, Scheme
import pandas as pd
import requests
from datetime import datetime, timedelta
import json

# Load data
url = "https://raw.githubusercontent.com/EisingerSyngenta/FloweringData/main/2022%20Locations.csv"
mydata = pd.read_csv(url)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Dropdown(id='rstcd-selector', placeholder='Select RSTCD'),
        dcc.Dropdown(id='loccd-selector', placeholder='Select LOCCD'),
        dcc.Dropdown(id='placd-selector', placeholder='Select PLACD'),
        dcc.DatePickerRange(
            id='date-range',
            min_date_allowed=datetime(2020, 1, 1),
            max_date_allowed=datetime.now().date() + timedelta(days=1),
            start_date=datetime(2020, 1, 1),
            end_date=datetime.now().date(),
        ),
        html.Button('Download GDU Data', id='download-button'),
        dcc.Download(id='download-gdu-csv')
    ], style={'width': '30%', 'display': 'inline-block', 'vertical-align': 'top'}),

    html.Div([
        dash_table.DataTable(
            id='gdu-table',
            columns=[
                {'name': 'Date', 'id': 'Date'},
                {'name': 'Daily GDU', 'id': 'DailyGDU', 'type': 'numeric', 'format': Format(precision=2, scheme=Scheme.fixed)},
                {'name': 'Cumulative GDU', 'id': 'CumulativeGDU', 'type': 'numeric', 'format': Format(precision=2, scheme=Scheme.fixed)}
            ],
            style_table={'height': '400px', 'overflowY': 'auto'},
            style_cell={'textAlign': 'left'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            }
        )
    ], style={'width': '70%', 'display': 'inline-block'})
])

@app.callback(
    Output('rstcd-selector', 'options'),
    Input('rstcd-selector', 'search_value')
)
def update_rstcd_options(search_value):
    return [{'label': i, 'value': i} for i in mydata['RSTCD'].unique()]

@app.callback(
    Output('loccd-selector', 'options'),
    Input('rstcd-selector', 'value')
)
def update_loccd_options(rstcd):
    available = mydata[mydata['RSTCD'] == rstcd]
    return [{'label': i, 'value': i} for i in available['LOCCD'].unique()]

@app.callback(
    Output('placd-selector', 'options'),
    [Input('rstcd-selector', 'value'),
     Input('loccd-selector', 'value')]
)
def update_placd_options(rstcd, loccd):
    available = mydata[(mydata['RSTCD'] == rstcd) & (mydata['LOCCD'] == loccd)]
    return [{'label': i, 'value': i} for i in available['PLACD'].unique()]

@app.callback(
    [Output('date-range', 'start_date'),
     Output('date-range', 'end_date')],
    [Input('rstcd-selector', 'value'),
     Input('loccd-selector', 'value'),
     Input('placd-selector', 'value')]
)
def update_date_range(rstcd, loccd, placd):
    if all([rstcd, loccd, placd]):
        available = mydata[(mydata['RSTCD'] == rstcd) & 
                           (mydata['LOCCD'] == loccd) & 
                           (mydata['PLACD'] == placd)]
        if not available.empty:
            planting_date = pd.to_datetime(available['planting_date'].iloc[0])
            return planting_date.date(), datetime.now().date()
    return dash.no_update, dash.no_update

@app.callback(
    Output('gdu-table', 'data'),
    [Input('rstcd-selector', 'value'),
     Input('loccd-selector', 'value'),
     Input('placd-selector', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_gdu_table(rstcd, loccd, placd, start_date, end_date):
    if all([rstcd, loccd, placd, start_date, end_date]):
        coords = mydata[(mydata['RSTCD'] == rstcd) & 
                        (mydata['LOCCD'] == loccd) & 
                        (mydata['PLACD'] == placd)][['Latitude', 'Longitude']]
        
        if coords.empty:
            return [{'Error': f'No coordinates found for RSTCD: {rstcd}, LOCCD: {loccd}, PLACD: {placd}'}]
        
        lat, long = coords.iloc[0]
        
        payload = {
            "units": {
                "temperature": "F",
                "velocity": "mph",
                "length": "imperial",
                "energy": "watts"
            },
            "geometry": {
                "type": "MultiPoint",
                "coordinates": [[long, lat]],
                "locationNames": [""]
            },
            "format": "json",
            "timeIntervals": [f"{start_date}T00:00:00Z/{end_date}T23:59:59Z"],
            "timeIntervalsAlignment": "none",
            "queries": [{
                "domain": "ERA5",
                "gapFillDomain": "NEMSGLOBAL",
                "timeResolution": "daily",
                "codes": [{
                    "code": 731,
                    "level": "2 m elevation corrected",
                    "aggregation": "sum",
                    "gddBase": 50,
                    "gddLimit": 86
                }]
            }]
        }
        
        print("Payload sent to API:")
        print(json.dumps(payload, indent=2))
        
        url = "http://my.meteoblue.com/dataset/query?apikey=syn82hw2eqe"
        try:
            response = requests.post(url, json=payload)
            response.raise_for_status()  # Raises an HTTPError for bad responses
            data = response.json()
        except requests.exceptions.RequestException as e:
            print(f"API Response: {response.text}")
            return [{'Error': f'API request failed: {str(e)}'}]
        
        if not data or 'error' in data:
            return [{'Error': f"API Error: {data.get('error_message', 'Unknown error')}"}]
        
        try:
            dates = [interval[:8] for interval in data[0]['timeIntervals'][0]]
            daily_gdu = data[0]['codes'][0]['dataPerTimeInterval'][0]['data'][0]
            
            # Convert dates to datetime objects for easier manipulation
            dates = [datetime.strptime(date, '%Y%m%d') for date in dates]
            
            # Filter data based on the selected date range
            start_date = datetime.strptime(start_date, '%Y-%m-%d')
            end_date = datetime.strptime(end_date, '%Y-%m-%d')
            
            filtered_data = [(date, gdu) for date, gdu in zip(dates, daily_gdu) if start_date <= date <= end_date]
            
            if not filtered_data:
                return [{'Error': 'No data available for the selected date range'}]
            
            dates, daily_gdu = zip(*filtered_data)
            
            cumulative_gdu = [sum(daily_gdu[:i+1]) for i in range(len(daily_gdu))]
            
            return [{'Date': date.strftime('%Y-%m-%d'), 'DailyGDU': round(daily, 2), 'CumulativeGDU': round(cumulative, 2)} 
                    for date, daily, cumulative in zip(dates, daily_gdu, cumulative_gdu)]
        except (KeyError, IndexError) as e:
            return [{'Error': f'Unexpected data format: {str(e)}'}]
    
    return []

@app.callback(
    Output('download-gdu-csv', 'data'),
    Input('download-button', 'n_clicks'),
    [State('rstcd-selector', 'value'),
     State('loccd-selector', 'value'),
     State('placd-selector', 'value'),
     State('gdu-table', 'data')],
    prevent_initial_call=True
)
def download_gdu_data(n_clicks, rstcd, loccd, placd, data):
    df = pd.DataFrame(data)
    return dcc.send_data_frame(df.to_csv, f"{rstcd}_{loccd}_{placd}_GDU.csv", index=False)

if __name__ == '__main__':
    app.run_server(debug=True)

Payload sent to API:
{
  "units": {
    "temperature": "F",
    "velocity": "mph",
    "length": "imperial",
    "energy": "watts"
  },
  "geometry": {
    "type": "MultiPoint",
    "coordinates": [
      [
        -96.65286641,
        46.92575737
      ]
    ],
    "locationNames": [
      ""
    ]
  },
  "format": "json",
  "timeIntervals": [
    "2024-05-08T00:00:00Z/2025-03-05T23:59:59Z"
  ],
  "timeIntervalsAlignment": "none",
  "queries": [
    {
      "domain": "ERA5",
      "gapFillDomain": "NEMSGLOBAL",
      "timeResolution": "daily",
      "codes": [
        {
          "code": 731,
          "level": "2 m elevation corrected",
          "aggregation": "sum",
          "gddBase": 50,
          "gddLimit": 86
        }
      ]
    }
  ]
}


In [10]:
!pip install dash pandas requests jupyter-dash


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
